#PyTorch Intro - Autoróżniczkowanie i Graf Obliczeń - Wykład

Do optymalizacji parametrów (wag) sieci neuronowej podczas treningu modelu wykorzystuje się **metodę stochastycznego spadku wzdłuż gradientu**.
Gradient funkcji straty względem parametrów sieci wyznaczany jest algorytmem **propagacji wstecznej** (*back propagation*).

Sieć neuronową możemy potraktować jak złożoną funkcję mapującą wejściowe dane $x \in \mathcal{X}$ (np. obraz czy sekwencję audio) na wyjście $y \in \mathcal{Y}$ parametryzowaną zestawem parametrów (wag) $\theta$.
$$
f_{\theta}( x ) = y
$$
W przypadku $n$-klasowego klasyfikatora wyjściem z sieci jest wektor $y \in \mathbb{R}^n$ nieznormalizowanych wartości, zwanych logitami, z których możemy wyznaczyć rozkład prawdopodobieństwa klas korzystając z funkcji softmax.

W jednym kroku treningu sieci neuronowych wykonujemy:
1. **Przejście w przód** - przetworzenie zestawu wejściowych danych treningowych przez sieć i wyznaczenie wartości wynikowych $y = f_{\theta}(x)$. Następnie wyznaczenie wartości funkcji straty
$\mathcal{L}$
w oparciu o wynikową wartość z sieci i prawdziwą (docelową) wartość.
2. **Przejście w tył** (propagacja wsteczna) - wyznaczenie **gradientu funkcji  straty** $\mathcal{L}$ **względem parametrów sieci** $\theta$.
3. Krok optymalizacji parametrów sieci - zmiana w kierunku przeciwnym do gradientu.

##Przygotowanie środowiska
Upewnij się, że notatnik jest uruchomiony na maszynie z GPU. Jeśli GPU nie jest dostępne zmień typ maszyny (Runtime | Change runtime type) i wybierz T4 GPU.

In [ ]:
!nvidia-smi

Biblioteka PyTorch (`torch`) jest domyślnie zainstalowana w środowisku COLAB.

In [ ]:
import torch
import numpy as np

print(f"Wersja biblioteki PyTorch: {torch.__version__}")

Sprawdzenie dostępnego urządzenia GPU.

In [ ]:
print(f"Dostępność GPU: {torch.cuda.is_available()}")
print(f"Typ GPU: {torch.cuda.get_device_name(0)}")

Instalacja pakietu torchviz do wizualizacji grafów obliczeń ([link](https://github.com/szagoruyko/pytorchviz)).

In [ ]:
!pip install -q torchviz

#Automatyczne różniczkowanie (`torch.autograd`)

**Gradient** (lub gradientowe pole wektorowe) funkcji skalarnej wielu zmiennych $
f: \mathbb{R}^D → \mathbb{R}
$ oznaczamyy
$\nabla f$ (czytaj: nabla).
W układzie współrzędnych kartezjańskich gradient jest wektorem, którego składowe są pochodnymi cząstkowymi funkcji $f$:
$$\nabla f=\left[{\frac {\partial f}{\partial x_{1}}},\dots ,{\frac {\partial f}{\partial x_{n}}}\right]$$

Niech $\mathcal{L}: \mathbb{R}^D \rightarrow \mathbb{R}$ będzie pewną funkcją straty określoną dla sieci neuronowej o $D$ parametrach (wagach).
Celem treningu sieci neuronowej jest znalezienie zestawu parametrów $\mathbf{\hat{}} \in \mathbb{R}^D$ minimalizującego wartośc funkcji straty:
$$\mathbf{\hat{w}} = \arg \min_{\textbf{w}} \mathcal{L} \left( \textbf{w} \right)$$
W metodzie **spadku wzdłuż gradientu** zaczynamy od losowo zainicjalizowanych parametrów (wag) sieci $\textbf{w}_0$ a następnie iteracyjnie aktualizujemy parametry sieci w kierunku przeciwnym do wartości gradientu:
$$
\mathbf{w}_{t+1} = \mathbf{w}_{t} - \eta \nabla \mathcal{L} \left( \mathbf{w}_t \right)
$$.





Aby wyznaczyć **gradient funkcji straty względem parametrów sieci**, PyTorch posiada wbudowany mechanizm różniczkowania o nazwie `torch.autograd`. Umożliwia on automatyczne obliczanie gradientu dla dowolnego grafu obliczeniowego.

Obiekty typu Tensor posiadają logiczną flagę `requires_grad`.
Domyślnie flaga `requires_grad` jest ustawiana na `False`.
Po jej włączeniu PyTorch będzie automatycznie budował grafy dla wszystkich obliczeń wykonanych z wykorzystaniem tego tensora aby umożliwić automatyczne wyznaczanie gradientu.
Jeśli jeden z argumentów operacji na tensorach ma ustawioną flagę `requires_grad`, wynik również będzie miał ustawioną tę flagę.

In [ ]:
import torch

x = torch.tensor([ 1., 2. ])
y = torch.tensor([ 4., 5. ])
z = torch.tensor([ 7., 3. ])
print(f"{x.requires_grad=}")
print(f"{y.requires_grad=}")
print(f"{(x + y).requires_grad=}\n")

x.requires_grad = True
print(f"{x.requires_grad=}")
print(f"{y.requires_grad=}")
print(f"{(x + z).requires_grad=}")

###Przykład 1
Rozważmy funkcję:
$$
f(t, u) = \sum_i t_i^2 + \sum_j \log u_j \, ,
$$
gdzie $t \in \mathbb{R}^n, u \in \mathbb{R}^m$.

Zaimplementujmy wyznaczenie wartości tej funkcji w PyTorch dla przykładowych parametrów $t=[1,2,4]$ i $u=[10,20]$. Aby było możliwe wyznaczenie wartości gradientu $f$ dla tych parametrów włączymy flagę `requires_grad`.

In [ ]:
t = torch.tensor([1., 2., 4.])
t.requires_grad = True
u = torch.tensor([10., 20.])
u.requires_grad = True

f = t.pow(2).sum() + u.log().sum()
print(f"{f=}")
print(f"{f.requires_grad=}")

Wizualizacja grafu obliczeń dla tensora `f`.

In [ ]:
from torchviz import make_dot
from IPython.display import Image, display

dot = make_dot(f, params={"t": t, "u": u, "f": f})
dot.render("computational_graph", format="png")
display(Image(filename="computational_graph.png"))

Metoda `f.backward()` wykonuje przejście w tył przez graf obliczeń w celu wyznaczenia gradientów względem parametrów (tensorów z włączoną flagą `requires_grad`) wykorzystanych do wyznaczenia wartości `f`. W naszym przypadku są to tensory `t` i `u`.

Pochodne cząstkowe funkcji $f$ względem parametrów $t$ i $u$ możemy wyznaczyć analitycznie:
$
\frac{\partial f}{\partial t_i} = 2 t_i
$ i
$
\frac{\partial f}{\partial u_i} = \frac{1}{u_i}
$.
Jak widać wyznaczone analitycznie wartości gradientu są identyczne jak wartości wyznaczone automatyczniez wykorzystaniem grafu obliczeń.


In [ ]:
print(f"\nPrzed wykonaniem f.backward()")
print(f"{t.grad=}")
print(f"{u.grad=}")

f.backward()
print(f"\nPo wykonaniu f.backward()")
print(f"{t.grad=}")
print(f"{u.grad=}")

##Przykład 2
Rozważmy najprostszą sieć neuronową złożoną z jednej warstwy liniowej z wejściem `x` $\in \mathbb{R}^{5}$ oraz pewną funkcję straty `loss`.
Warstwa liniowa jest zaimplementowana w PyTorch jako klasa `nn.Linear` ([link](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#linear)).

Do celów poglądowych zasymulujemy działanie warstwy linowej korzystając z macierzy wag `w` $\in \mathbb{R}^{5 \times 3}$ i wektora obciążenia (*bias*) `b` $\in \mathbb{R}^{3}$.
`w` i `b` są **parametrami**, które chcemy optymalizować w procesie uczenia sieci. W tym celu musimy wyznaczyć gradient funkcji straty względem tych parametrów. Aby umożliwić automatyczne wyznaczanie gradientu dla tych parametrów, ustawiamy dla nich atrybut `requires_grad` na `true`.
Zauważmy, że tensory utworzone w oparciu o choćby jeden tensor z atrybutem `requires_grad`, w tym przypadku `z` i `loss`, również będą miały aktywne wyznaczanie gradientu.

**Uwaga**: Jesli wykorzystujemy moduły sieci dziedziczące z klasy `nn.Module` (np. `nn.Linear`, `nn.Conv2D`) tensory parametrów (wag) mają atrybut `requires_grad` domyślnie ustawiony na `true`.

In [ ]:
import torch.nn as nn

x = torch.ones(5)           # Tensor wejściowy
y = torch.zeros(3)          # Oczekiwane wyjście

# Parametry warstwy liniowej: macierz wag i wektor obciążenia (bias)
w = torch.randn((5, 3), requires_grad=True)
b = torch.randn(3, requires_grad=True)

z = x @ w + b

# Wyznaczmy wartość funkcji straty jako funkcję entropii krzyżowej między
# wyjściem z sieci a oczekiwanym wyjściem
loss = nn.functional.binary_cross_entropy_with_logits(z, y)

In [ ]:
print(f"{z=}")
print(f"{z.requires_grad=}")
print(f"{loss=}")
print(f"{loss.requires_grad=}")

###Wizualizacja grafu obliczeń



In [ ]:
from torchviz import make_dot
from IPython.display import Image, display

dot = make_dot(loss, params={"w": w, "b": b, "loss": loss})
dot.render("computational_graph", format="png")
display(Image(filename="computational_graph.png"))

Funkcje i operatory które stosujemy do tensorów w celu skonstruowania grafu obliczeń, takie jak operatory dodawania i mnożenia dwóch tensorów czy funkcja `binary_cross_entropy_with_logits`, są zaimplementowne jako klasy dziedziczące z klasy `Function`. Mają zdefiniowaną metodę `forward` wyznaczającą wartość operatora/funkcji oraz metodę `backward` wyznaczającą jej pochodną w kroku propagacji wstecznej.
Odwołanie do funkcji propagacji wstecznej tensora jest przechowywane w atrybucie `grad_fn`.
Więcej informacji na temat klasy Function można znaleźć tutaj: [link](https://www.google.com/url?q=https%3A%2F%2Fpytorch.org%2Fdocs%2Fstable%2Fautograd.html%23function).

In [ ]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

###Obliczanie gradientu

Aby wykonać jeden krok optymalizacji wag sieci musimy wyznaczyć gradient funkcji straty względem parametrów sieci, czyli $\frac{\partial loss}{\partial w}$ i
$\frac{\partial loss}{\partial b}$ dla ustalonych wartości `x` i
`y`.
Aby wyznaczyć te pochodne cząstkowe wywołujemy funkcję `loss.backward()`. Wyznaczone pochodne zostaną zapisane w atrybutach `w.grad` and `b.grad`.

In [ ]:
loss.backward()
print(f"{w.grad=}")
print(f"{b.grad=}")

Zasymulujemy teraz jeden krok optymalizacji parametrów sieci metodą spadku wzdłuż gradientu.
Gradient funkcji straty `loss` względem parametrów (macierz wag `w` i wektor obciążeń `b`) wyznaczony w poprzednim kroku określa kierunek maksymalnego wzrostu wartości funkcji straty.
Ponieważ chcemy minimalizować wartość funkcji straty, musimy zmieniać parametry w kierunku przeciwnym do gradientu.

Graf obliczeń powinien być budowany tylko dla obliczeń prowadzących do wyznaczenia wartości funkcji straty.
Przy samej aktualizacji parametrów sieci z wykorzystaniem wyznaczonych gradientów powinniśmy wyłączyć budowanie grafu obliczeń korzystając z menadżera kontekstu `torch.no_grad():`.


In [ ]:
print(f"Początkowa wartość funkcji straty: {loss:.5f}")

learning_rate = .5
with torch.no_grad():
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad

z = x @ w + b
loss2 = nn.functional.cross_entropy(z, y)

print(f"Wartość funkcji straty po jednym kroku optymalizacji parametrów: {loss2:.5f}")

**Uwaga:** Ze względów wydajnościowych wyznaczone pochodne są zapisywane tylko dla liści grafu obliczeń z ustawionym atrybutem `requires_grad`. Dla wewnętrznych węzłów w grafie obliczeń (w naszym przypadku `z` i `loss`) atrybut `grad` nie jest zapisywany.

In [ ]:
print(f"{z.grad=}")
print(f"{loss.grad=}")

Zapisywanie wartości wyznaczonego gradientu dla wewnętrznych węzłów w grafie obliczeń można włączyć wywołując dla nich metodę `retain_grad()`.

In [ ]:
# Parametry warstwy liniowej: macierz wag i wektor obciążenia (bias)
w = torch.randn((5, 3), requires_grad=True)
b = torch.randn(3, requires_grad=True)

z = x @ w + b
z.retain_grad()

# Wyznaczmy wartość funkcji straty jako funkcję entropii krzyżowej między
# wyjściem z sieci a oczekiwanym wyjście
loss = nn.functional.binary_cross_entropy_with_logits(z, y)
loss.retain_grad()

loss.backward()
print(f"{z.grad=}")
print(f"{loss.grad=}")
print(f"{w.grad=}")
print(f"{b.grad=}")

##Przykład 3
Funkcja
$$\mathcal{L}=\sqrt{sin \left( x \cdot y + z \right) + 4}$$



In [ ]:
x = torch.tensor(2.)
x.requires_grad = True
y = torch.tensor(-1.)
y.requires_grad = True
z = torch.tensor(2.)
z.requires_grad = True

L = torch.sqrt(torch.sin(x * y + z) + 4)

print(f"{L=}")
print(f"{L.requires_grad=}")

In [ ]:
L.backward()
print(f"{x.grad=}")
print(f"{y.grad=}")
print(f"{z.grad=}")

In [ ]:
dot = make_dot(L, params={"x": x, "y": y, "z": z, "L": L})
dot.render("computational_graph", format="png")
display(Image(filename="computational_graph.png"))

##Wyłączenie śledzenia historii obliczeń

Historia obliczeń wykonywanych z wykorzystaniem tensorów z atrybutem `requires_grad=True` jest śledzona i automatycznie budowany jest graf obliczeń.  Tensory będące wynkiem takich obliczeń również mają ustawiony atrybut `requires_grad` i historia obliczeń z ich wykonaniem jest dalej śledzona.

W niektórych przypadkach nie jest to konieczne. Jeśli chcemy wykorzystać wcześniej wytrenowany model, i nie zamierzamy optymalizować jego parametrów, możemy wyłączyć śledzenie obliczeń korzystając z menadżera kontekstu `torch.no_grad()`.

Alternatywnie możemy ustawić atrybut `requires_grad` wszystkich lub wybranych parametrów modelu na `False`. W ten sposób można zamrozić część parametrów modelu i optymalizować tylko pozostałe parametry.

In [ ]:
x = torch.ones(5)

z = x @ w + b
print(f"{z.requires_grad=}")

with torch.no_grad():
    z = x @ w + b

print(f"{z.requires_grad=}")

Alternatywnie możemy użyć metodę `detach()` aby odłączyć graf obliczeń od istniejącego tensora. Tak uzyskany tensor będzie miał wyłączony atrybut `requires_grad`. Historia obliczeń z jego udziałem nie będzie śledzona i nie będzie możliwe wyznaczenie dla niego gradientu.

In [ ]:
z = x @ w + b
print(f"{z.requires_grad=}")
y = z * 3 + 2
print(f"{y.requires_grad=}")
print()

z_det = z.detach()
print(f"{z_det.requires_grad=}")
y_det = z_det * 3 + 2
print(f"{y_det.requires_grad=}")

#Przykład: Regresja logistyczna
Rozważny problem dwuklasowej regresji logistycznej:
$$P(Y=1|X=x) = \sigma(W \cdot x + b) \, ,$$
gdzie $\sigma(x)$ jest **funkcją sigmoidalną** daną wzorem: $$\sigma(x) = \frac{1}{1+\exp(-x)} \, ,$$
a zmienna zależna $Y$ przyjmuje wartości ze zbioru $\left\{ 0, 1 \right\}$.

Funkcja straty binarnej entropii krzyżowej między prawdziwym a estymowanym prawdopodobieństwem dana jest wzorem:
$$
\mathcal{L} = y_i \log \sigma \left(z_i\right) + (1 - y_i) \log
\left( 1 - \sigma \left(z_i\right)  \right) \, ,
$$
gdzie
$$
z_i = W \cdot x_i + b
$$

Problem ten nie ma rozwiązania analitycznego. Wartość parametrów (macierz wag $W$ i wektor obciążenia $b$) możemy znaleźć metodą spadku wzdłuż gradientu.

Wygenerujmy syntetyczny zbiór danych korzystając z funkcji `make_classification` z biblkioteki scikit-learn ([link](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html)).

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Wygeneruj syntetyczny zbiór danych
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, n_informative=5, random_state=39)

# Podziel na dane treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(f"Liczba elementów w zbiorze danych: {X.shape[0]}")
print(f"Liczba cech: {X.shape[1]}")
print(f"Liczba klas: {len(np.unique(y))}")

# Podziel na częśc treningową (80%) i testową (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(X_train[:6])
print(y_train[:6])

Wizualizacja dwóch wybranych cech. Możemy zauważyć, że dla dwóch losowo wybranych cech elementy z różnych klas nie są liniowo separowalne.

In [ ]:
import random
from matplotlib import pyplot as plt

# Wybierz dwie losowe cechy
f1 = random.randint(0, X_train.shape[1]-1)
f2 = random.randint(0, X_train.shape[1]-1)

plt.scatter(X_train[y_train==0, f1], X_train[y_train==0, f2], color='blue', label='Klasa 0', alpha=0.7)
plt.scatter(X_train[y_train==1, f1], X_train[y_train==1, f2], color='red', label='Klasa 1', alpha=0.7)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xlabel(f"Wartość cechy {f1}")
plt.ylabel(f"Wartość cechy {f2}")
plt.legend()
plt.show()


Do wyznaczenia wartości wyrażenia $z_i = W  x_i + b$ wykorzystamy **warstwę liniową** zaimplementowaną w bibliotece PyTorch jako moduł sieci `nn.Linear` ([link](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)).
Utworzymy warstwę liniową o wymiarze wejścia $=10$ i wymiarze wyjścia $=1$ (czyli na wejściu będzie wektor $x_i \in \mathbb{R}^{10}$ a na wyjściu skalar $z_i \in \mathbb{R}$).

Zauważmy, że utworzona warstwa liniowa (obiekt `linear`) ma dwa parametry: macierz wag `linear.weight` i wektor obciążenia `linear.bias`. Każdy z nich jest tensorem o losowo zainicjalizowanych wartościach z włączonym śledzeniem historii obliczeń (`requires_grad=True`).

In [ ]:
import torch.nn as nn

n_features = X_train.shape[1]
linear = nn.Linear(n_features, 1)

print(f"{linear=}\n")
print(f"{linear.weight=}\n")
print(f"{linear.bias=}")

Sprawdźmy działanie warstwy liniowej.
Po przetworzeniu wektora cech $x \in \mathbb{R}^{10}$ w rezultacie otrzymamy pojedyczną wartość rzeczywistą (logit).
Warstwa liniowa pozwoli również przetworzyć cały wsad wektorów cech $X \in \mathbb{R}^{N \times 10}$ i w rezultacie otrzymamy wektor $N$ logitów.
Zauważmy, że tensor `z` zawierający wynikowe wartości ma również włączone śledzenie obliczeń (`requires_grad=True`).


In [ ]:
# Zamień dane (jako tablice ndarray) na tensory
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

elem_ndx = 10
z = linear(X_train_tensor[elem_ndx])
print(f"Rozmiar wejścia: {X_train_tensor[elem_ndx].shape}")
print(f"Rozmiar wyjścia: {z.shape}")
print(f"Wyjście: {z}")
print(f"{z.requires_grad=}")
print()

z = linear(X_train_tensor)
print(f"Rozmiar wejścia: {X_train_tensor.shape}")
print(f"Rozmiar wyjścia: {z.shape}")
print(f"Wyjście (pierwsze 5 elementów): {z[:5]}")
print(f"{z.requires_grad=}")

Optymalizacja parametrów (wag) warstwy liniowej metodą spadku wzdłuż gradientu.
Do obliczania wartości straty wykorzystamy funkcję `BCEWithLogitsLoss`, łączącą w jednej klasie funkcję Sigmoid i funkcję straty binarnej entropii krzyżowej (patrz: [link](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html#torch.nn.BCEWithLogitsLoss)).
Pozwala to na bardziej stabilne numerycznie obliczenie niż zastosowanie najpierw funkcji Sigmoid a następnie wyznaczenie straty binarnej entropii krzyżowej.

W literaturze poświęconej głębokiemu uczeniu nieznormalizowane wyjścia z modułu sieci, w naszym przypadku z warstwy liniowej, mogące przyjmować wartości rzeczywiste z zakresu $\left( -\infty, \infty \right)$ zwane są **logitami**.

**Uwaga:** Poniższy kod ma charakter poglądowy. W praktyce optymalizację parametrów (wag) sieci neuronowej wykonujemy korzystając z zaimplementowanego w PyTorch optymalizatora (np. SGD lub Adam).

In [ ]:
criterion = nn.BCEWithLogitsLoss()

eta= 0.1
n_epochs = 200
for k in range(n_epochs):
    # Wyzeruj wartości gradientu przed kolejnym przejściem w tył
    # Patrz: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
    linear.zero_grad()

    logits = linear(X_train_tensor)
    # logits ma rozmiar (N, 1)
    logits = logits.squeeze(1)
    # logits ma rozmiar (N,)

    # Wyznacz wartość funkcji straty
    loss = criterion(logits, y_train_tensor)

    # Przejście w tył- wyznaczenie gradientu funkcji straty względem parametrów (wag) modelu
    loss.backward()

    # Krok optymalizacji parametrów sieci - zmiana w kierunku przeciwnym do wartości gradientu
    with torch.no_grad():
        linear.weight -= eta * linear.weight.grad
        linear.bias -= eta * linear.bias.grad

    # Dokładność klasyfikacji na zbiorze treningowym
    y_pred_labels = (logits.detach() >= 0.5).float()
    train_accuracy = (y_pred_labels == y_train_tensor).sum().item() / y_train_tensor.size(0)

    # Dokładność klasyfikacji na zbiorze testowym
    with torch.no_grad():
        logits = linear(X_test_tensor)
        logits = logits.squeeze(1)
        y_pred_labels = (logits >= 0.5).float()
        test_accuracy = (y_pred_labels == y_test_tensor).sum().item() / y_test_tensor.size(0)

    if k % 20 == 0:
        print(f"Epoch: {k}   Wartość funkcji straty: {loss.item():.5f}   Dokładność (train): {train_accuracy:.4f}   Dokładność (test): {test_accuracy:.4f}")

Dla porównania sprawdzimy jaką skuteczność osiąga klasyfikator regresji logistycznej z biblioteki scikit-learn.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

# Step 5: Evaluate the model
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Dokładność (train): {train_accuracy:.4f}   Dokładność (test): {test_accuracy:.4f}")